In [23]:
import csv

# Đọc dữ liệu giao dịch từ tệp CSV
def read_transactions_from_csv(file_path):
    transactions = []
    with open(file_path, newline='', encoding='utf-8-sig') as csvfile:  # Dùng utf-8-sig để xử lý BOM
        reader = csv.reader(csvfile)
        for row in reader:
            cleaned_row = {item.strip() for item in row if item.strip()}  # Loại bỏ mục rỗng và khoảng trắng
            if cleaned_row:  # Bỏ qua các dòng trống hoàn toàn
                transactions.append(cleaned_row)
    return transactions


In [24]:


# Ngưỡng
minsup = 0.3
minconf = 1.0

# Hàm tính độ hỗ trợ cho một tập hợp
def calculate_support(transactions, itemset):
    count = sum(1 for transaction in transactions if itemset.issubset(transaction))
    return count / len(transactions)

# Hàm sinh tập phổ biến tối đại
def find_frequent_itemsets(transactions, minsup):
    items = {item for transaction in transactions for item in transaction}
    level_1 = [{item} for item in items if calculate_support(transactions, {item}) >= minsup]
    frequent_itemsets = [set(itemset) for itemset in level_1]
    all_frequent = [frequent_itemsets]
    
    k = 2
    while True:
        # Sinh ứng viên cấp k
        candidates = []
        previous_level = all_frequent[-1]
        for i in range(len(previous_level)):
            for j in range(i + 1, len(previous_level)):
                union_set = previous_level[i] | previous_level[j]
                if len(union_set) == k and union_set not in candidates:
                    candidates.append(union_set)
        # Lọc tập phổ biến
        current_level = [c for c in candidates if calculate_support(transactions, c) >= minsup]
        if not current_level:
            break
        all_frequent.append(current_level)
        frequent_itemsets.extend(current_level)
        k += 1
    
    # Lọc tập phổ biến tối đại
    maximal_itemsets = []
    for itemset in frequent_itemsets:
        if not any(itemset < other for other in frequent_itemsets):  # Kiểm tra không có tập cha nào phổ biến hơn
            maximal_itemsets.append(itemset)
    
    return maximal_itemsets

# Hàm sinh luật kết hợp từ tập phổ biến tối đại
def generate_association_rules(maximal_itemsets, transactions, minconf):
    rules = []
    for itemset in maximal_itemsets:
        subsets = [set(sub) for sub in generate_subsets(itemset)]
        for antecedent in subsets:
            consequent = itemset - antecedent
            if antecedent and consequent:
                support_itemset = calculate_support(transactions, itemset)
                support_antecedent = calculate_support(transactions, antecedent)
                confidence = support_itemset / support_antecedent if support_antecedent > 0 else 0
                if confidence >= minconf:
                    rules.append((antecedent, consequent, confidence))
    return rules

# Sinh tất cả tập hợp con của một tập hợp
def generate_subsets(itemset):
    items = list(itemset)
    subsets = []
    for i in range(1, 1 << len(items)):  # Từ 1 đến 2^n - 1
        subset = {items[j] for j in range(len(items)) if (i & (1 << j))}
        subsets.append(subset)
    return subsets



In [25]:
# Đường dẫn tới tệp CSV
file_path = 'D:\hk5\khai thác dữ liệu\DOAN\dta.csv'


<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15672\3391092964.py:2: SyntaxWarning: invalid escape sequence '\h'
  file_path = 'D:\hk5\khai thác dữ liệu\DOAN\dta.csv'


In [26]:
# Đọc giao dịch từ tệp CSV
transactions = read_transactions_from_csv(file_path)


In [27]:
# Thực hiện thuật toán
maximal_itemsets = find_frequent_itemsets(transactions, minsup)
association_rules = generate_association_rules(maximal_itemsets, transactions, minconf)

maximal_itemsets, association_rules


([{'i5', 'i6'},
  {'i1', 'i3', 'i6'},
  {'i1', 'i4', 'i6'},
  {'i2', 'i4', 'i6'},
  {'i1', 'i2', 'i4', 'i5'}],
 [({'i1', 'i3'}, {'i6'}, 1.0),
  ({'i2', 'i6'}, {'i4'}, 1.0),
  ({'i2', 'i5'}, {'i1', 'i4'}, 1.0),
  ({'i4', 'i5'}, {'i1', 'i2'}, 1.0),
  ({'i2', 'i4', 'i5'}, {'i1'}, 1.0),
  ({'i1', 'i2'}, {'i4', 'i5'}, 1.0),
  ({'i1', 'i2', 'i4'}, {'i5'}, 1.0),
  ({'i1', 'i5'}, {'i2', 'i4'}, 1.0),
  ({'i1', 'i2', 'i5'}, {'i4'}, 1.0),
  ({'i1', 'i4', 'i5'}, {'i2'}, 1.0)])

In [28]:
# Lọc tập phổ biến
maximal_itemsets = [itemset for itemset in maximal_itemsets if '' not in itemset]

# Lọc luật kết hợp
association_rules = [
    (antecedent, consequent, confidence)
    for antecedent, consequent, confidence in association_rules
    if '' not in antecedent and '' not in consequent
]


In [29]:
print("Dữ liệu giao dịch sau khi làm sạch:")
for transaction in transactions:
    print(transaction)


Dữ liệu giao dịch sau khi làm sạch:
{'i6', 'i3', 'i4', 'i1'}
{'i6', 'i3', 'i1'}
{'i6', 'i3', 'i5'}
{'i2', 'i4', 'i5', 'i1'}
{'i2', 'i6', 'i4'}
{'i6', 'i1', 'i2', 'i4', 'i5'}


In [30]:
# In kết quả
print("Tập phổ biến tối đại:")
for itemset in maximal_itemsets:
    print(itemset)

print("\nCác luật kết hợp:")
for antecedent, consequent, confidence in association_rules:
    print(f"{antecedent} -> {consequent} (confidence = {confidence:.2f})")

Tập phổ biến tối đại:
{'i6', 'i5'}
{'i6', 'i3', 'i1'}
{'i6', 'i4', 'i1'}
{'i2', 'i6', 'i4'}
{'i2', 'i4', 'i5', 'i1'}

Các luật kết hợp:
{'i3', 'i1'} -> {'i6'} (confidence = 1.00)
{'i2', 'i6'} -> {'i4'} (confidence = 1.00)
{'i2', 'i5'} -> {'i4', 'i1'} (confidence = 1.00)
{'i4', 'i5'} -> {'i2', 'i1'} (confidence = 1.00)
{'i2', 'i4', 'i5'} -> {'i1'} (confidence = 1.00)
{'i2', 'i1'} -> {'i4', 'i5'} (confidence = 1.00)
{'i2', 'i4', 'i1'} -> {'i5'} (confidence = 1.00)
{'i5', 'i1'} -> {'i2', 'i4'} (confidence = 1.00)
{'i2', 'i5', 'i1'} -> {'i4'} (confidence = 1.00)
{'i4', 'i5', 'i1'} -> {'i2'} (confidence = 1.00)
